In [1]:
import requests

# Get the page
response = requests.get("https://www.aucklandcouncil.govt.nz/rubbish-recycling/rubbish-recycling-collections/Pages/collection-day-detail.aspx?an=12340955518")
rubbish_page = response.text


In [55]:
import re

rubbish_text = re.sub(r"\s+$", "", # Kill trailing whitespace
                      re.sub(r"^\s+", "", # Kill leading whitespace
                             re.sub(r"\n\s*\n", "\n", # Merge multi-line breaks
                                    re.sub(r"<[^>]*>", " ", rubbish_page)), flags=re.M), flags=re.M) # Remove html tags

# Grab the lines
match = re.search(r"Your next collection dates:\n^(.*?)$\n^(.*?)$", rubbish_text, flags=re.M)

In [56]:
texts = [match.group(1), match.group(2)]
rubbish_dates = []

for text in texts:
    text_match = re.search(r"^([\w\s]+?)\s{2,}(.*)$", text, re.M)
    raw_date = text_match.group(1)
    raw_rubbish_flags = text_match.group(2).lower()

    date_match = re.search(r"^(\w+)\s(\d+)\s(\w+)$", raw_date, re.M)
    raw_day = date_match.group(2)
    raw_month = date_match.group(3) 

    is_recycle = "recycle" in raw_rubbish_flags
    is_food_scraps = "food scraps" in raw_rubbish_flags
    is_waste = "rubbish" in raw_rubbish_flags
    rubbish_dates.append((raw_date, raw_rubbish_flags, is_recycle, is_food_scraps, is_waste))

In [57]:
rubbish_dates

[('Tuesday 4 July', 'rubbish     food scraps', False, True, True),
 ('Tuesday 11 July', 'rubbish     food scraps     recycle', True, True, True)]

In [2]:
# Find the URL for rubbish collection
BASE_URL = "https://www.aucklandcouncil.govt.nz/rubbish-recycling/rubbish-recycling-collections/Pages/collection-day-detail.aspx?an="
BASE_API_URL = "https://www.aucklandcouncil.govt.nz/_vti_bin/ACWeb/ACservices.svc/GetMatchingPropertyAddresses"
addr_response = requests.post(BASE_API_URL, json={
    "RateKeyRequired": False,
    "ResultCount": "1",
    "SearchText": "10 queen street"
})

In [8]:
addr_response.json()

[{'ACRateAccountKey': '12343324624',
  'Address': '10 Queen Street, Papakura',
  'Suggestion': '10 Queen Street, Papakura'}]

In [3]:
addr_response.status_code

200